# Profiling MNIST example with 3-conv. layer network

This example demonstrates the usage of `HandlersTimeProfiler`. The example uses MNIST dataset.

## Install requirements

In [2]:
!which python

/home/perceptron/.pyenv/versions/ignite/bin/python


## Imports

In [1]:
%matplotlib inline

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import MNIST

# A hack to fix the horizontal spill in large output
# ref: https://stackoverflow.com/a/59058418/6574605
from IPython.core.display import HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

## Loading MNIST

In [2]:
mnist_pwd = "data"
batch_size= 256

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

trainset = MNIST(mnist_pwd, train=True, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=4)

testset = MNIST(mnist_pwd, train=False, download=True, transform=transform)
testloader = DataLoader(testset, batch_size=batch_size * 2, shuffle=False, num_workers=0)

## Model

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [1]:
from ignite.engine import Events, create_supervised_trainer, create_supervised_evaluator
from ignite.metrics import Loss, Accuracy
from ignite.contrib.handlers import ProgressBar, HandlersTimeProfiler

## Training Loss

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
criterion = nn.NLLLoss()
model = Net()
model.to(device)  # Move model before creating optimizer
optimizer = optim.SGD(model.parameters(), lr=3e-4, momentum=0.9)

In [ ]:
trainer = create_supervised_trainer(model, optimizer, criterion, device=device)
evaluator = create_supervised_evaluator(model, metrics={"acc": Accuracy(), "loss": Loss(nn.NLLLoss())}, device=device)

# Attach handlers profiler
profiler = HandlersTimeProfiler()
profiler.attach(trainer)

# Init and attach progressbar
pbar = ProgressBar(persist=True)
pbar.attach(trainer, metric_names="all")

# Evaluate on each epoch using event handler
@trainer.on(Events.EPOCH_COMPLETED)
def log_validation_results(engine):
    evaluator.run(testloader)
    metrics = evaluator.state.metrics
    avg_accuracy = metrics["acc"]
    avg_nll = metrics["loss"]
    pbar.log_message(
        "Validation Results - Epoch: {}  Avg accuracy: {:.2f} Avg loss: {:.2f}".format(
            engine.state.epoch, avg_accuracy, avg_nll
        )
    )

    pbar.n = pbar.last_print_n = 0

trainer.run(trainloader, max_epochs=10)


Validation Results - Epoch: 1  Avg accuracy: 0.44 Avg loss: 2.19


We can see the summary of the profiling results from our run using the `get_results()` method of the profiler as shown below. The output shows total, average and other details of execution time for each handler attached.

In [ ]:
profiler.print_results(profiler.get_results())

Profiling results can be exported to a CSV file by using the `write_results()` method of profiler.

In [ ]:
profiler.write_results("./results.csv")

Following code shows the preview of few rows of the CSV. Each handler has its separate column and the numbers of rows for each column will be equal to the number of times the handler invoked.

In [ ]:
import pandas as pd

In [ ]:
results = pd.read_csv("./results.csv")
results.head()